In [ ]:
import requests
import sqlite3
import json
import pandas as pd
import base64

# SQLite 데이터베이스 연결 및 테이블 생성
try:
    conn = sqlite3.connect('cocacola_data.db')
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS store_items (
            id TEXT PRIMARY KEY,
            type TEXT,
            goods_seq INTEGER,
            goods_name TEXT,
            goods_image TEXT,
            content TEXT,
            tags TEXT,
            member_seq INTEGER,
            images TEXT,
            status TEXT,
            likes INTEGER,
            best BOOLEAN,
            best_at TEXT,
            created_at TEXT,
            updated_at TEXT,
            nickname TEXT,
            grade TEXT,
            href TEXT,
            mine BOOLEAN
        )
    ''')
except sqlite3.Error as e:
    print(f"SQLite 데이터베이스 오류 발생: {e}")
    exit()

# 데이터 수집 함수
def collect_data(offset):
    url = "https://cokeplay-api.cocacola.co.kr/community/search"
    params = {
        'limit': 10,
        'q': '',
        'sort': 'likes',
        'type': 'store',
        'offset': offset
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
    except requests.exceptions.RequestException as e:
        print(f"데이터를 가져오는 중 오류 발생: {e}")
        return None

    return data

# 데이터 삽입 함수
def insert_data(items):
    try:
        for item in items:
            cursor.execute('''
                INSERT OR IGNORE INTO store_items (
                    id, type, goods_seq, goods_name, goods_image, content, tags, member_seq, images, status,
                    likes, best, best_at, created_at, updated_at, nickname, grade, href, mine
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                item.get('id'),
                item.get('type'),
                item.get('goods_seq'),
                item.get('goods_name'),
                item.get('goods_image'),
                item.get('content'),
                json.dumps(item.get('tags', [])),
                item.get('member_seq'),
                json.dumps(item.get('images', [])),
                item.get('status'),
                item.get('likes'),
                item.get('best', False),
                item.get('best_at'),
                item.get('created_at'),
                item.get('updated_at'),
                item.get('nickname'),
                item.get('grade'),
                item.get('href'),
                item.get('mine', False)
            ))

        # 페이지에서 하나의 리뷰를 샘플로 출력
        if items:
            sample_review = items[0]  # 첫 번째 리뷰를 샘플로 선택
            print(f"샘플 리뷰 (ID: {sample_review['id']}):")
            print(f"상품명: {sample_review['goods_name']}")
            print(f"리뷰 내용: {sample_review['content']}")
            print(f"좋아요 수: {sample_review['likes']}")
            print("-" * 40)

    except sqlite3.Error as e:
        print(f"데이터 삽입 중 오류 발생: {e}")
        conn.rollback()

# 데이터 수집 중단 시 DB에서 데이터를 읽어오는 함수
def load_data_from_db():
    try:
        df = pd.read_sql_query("SELECT * FROM store_items", conn)
        print("데이터를 성공적으로 불러왔습니다.")
        return df
    except sqlite3.Error as e:
        print(f"데이터베이스 쿼리 오류 발생: {e}")
        return None

# 데이터 수집 및 저장
offset = "Mw=="  # 초기 offset 설정
while True:
    data = collect_data(offset)
    if not data or not data['items']:
        break
    insert_data(data['items'])
    # offset 값을 인코딩
    new_offset = str(int(base64.b64decode(offset).decode()) + 10)
    offset = base64.b64encode(new_offset.encode()).decode()

    if not offset:  # 다음 페이지가 없으면 종료
        break

conn.commit()

# 데이터베이스 연결 종료
conn.close()

# 중단 시 데이터 로드 함수 사용 예시
# 필요한 시점에 호출하여 DB에서 현재까지 수집된 데이터를 불러올 수 있음
df = load_data_from_db()
if df is not None:
    print(df.head())  # 불러온 데이터의 일부를 출력


In [ ]:
df = load_data_from_db()
df

In [ ]:
# prompt: df 변수에 content 컬럼에 중복 데이터가 몇개 있는지와 어떤 데이터가 중복인지 확인하는 코드 작성

if df is not None:
  duplicate_content = df[df.duplicated(subset=['id'], keep=False)]
  num_duplicates = len(duplicate_content)

  print(f"중복된 content 데이터 개수: {num_duplicates}")
  print("중복된 데이터:")
  duplicate_content